In [ ]:
# todo: check the model cfg: augmentations, policy

In [1]:
import torch
from torch import nn
from utils import parse_cfg
from darknet import Darknet

In [2]:
darknet = Darknet('cfg/yolov3.cfg')

to del, this message should be printed 3ish times
to del, this message should be printed 3ish times
to del, this message should be printed 3ish times
make_layers returns net_info as well. check whether it"s necessary
shortcut is using output[i-1] instead of x check whether works with x
NOTE THAT CONV BEFORE YOLO USES (num_classes filters) * num_anch


In [5]:
darknet.filters_cache

[3,
 32,
 64,
 32,
 64,
 128,
 64,
 128,
 64,
 128,
 256,
 128,
 256,
 128,
 256,
 128,
 256,
 128,
 256,
 128,
 256,
 128,
 256,
 128,
 256,
 128,
 256,
 512,
 256,
 512,
 256,
 512,
 256,
 512,
 256,
 512,
 256,
 512,
 256,
 512,
 256,
 512,
 256,
 512,
 1024,
 512,
 1024,
 512,
 1024,
 512,
 1024,
 512,
 1024,
 512,
 1024,
 512,
 1024,
 512,
 1024,
 255,
 1024,
 256,
 512,
 256,
 512,
 256,
 512,
 256,
 512,
 255,
 512,
 128,
 384,
 128,
 256,
 128,
 256,
 128,
 256,
 255]

In [31]:
del predictions

In [35]:
x = torch.rand(1, 13, 85)

try:
    predictions = torch.cat((predictions, x), dim=1)
        
except NameError:
    predictions = x
    
print(predictions)
l = torch.nn.Conv1d(13, 2, 3)
x = l(x)
print(predictions)

tensor([[[0.9272, 0.8688, 0.7273,  ..., 0.9769, 0.3683, 0.3115],
         [0.2115, 0.6499, 0.5213,  ..., 0.9838, 0.2816, 0.5502],
         [0.0281, 0.3653, 0.7759,  ..., 0.1455, 0.4885, 0.3471],
         ...,
         [0.4669, 0.1822, 0.6258,  ..., 0.0174, 0.5350, 0.4395],
         [0.6263, 0.0082, 0.5887,  ..., 0.4463, 0.3623, 0.6832],
         [0.8101, 0.8784, 0.3168,  ..., 0.1351, 0.2711, 0.4058]]])
tensor([[[0.9272, 0.8688, 0.7273,  ..., 0.9769, 0.3683, 0.3115],
         [0.2115, 0.6499, 0.5213,  ..., 0.9838, 0.2816, 0.5502],
         [0.0281, 0.3653, 0.7759,  ..., 0.1455, 0.4885, 0.3471],
         ...,
         [0.4669, 0.1822, 0.6258,  ..., 0.0174, 0.5350, 0.4395],
         [0.6263, 0.0082, 0.5887,  ..., 0.4463, 0.3623, 0.6832],
         [0.8101, 0.8784, 0.3168,  ..., 0.1351, 0.2711, 0.4058]]])


In [66]:
del x, predictions

In [72]:
# todo forward use both info and modules 
x = torch.randn((2, 3, 608, 608))
device = 'cpu'

self_layer_list = darknet.layers_list
self_layer_info = darknet.layers_info[1:]

# cache the outputs for route and shortcut layers
outputs = []

for i, layer in enumerate(self_layer_list):
    name = self_layer_info[i]['name']
    print(i, name)
    # TODO: append something after each layer!!!!
    if name in ['convolutional', 'upsample']:
        print(x.shape, layer)
        x = layer(x)
        
    elif name == 'shortcut':
        # index which is used for shortcut (usually '-3')
        x = outputs[-1] + outputs[layer[0].frm]
#         x = x + outputs[layer.frm]
        
    elif name == 'route':
        to_cat = [outputs[route_idx] for route_idx in layer[0].routes]
        x = torch.cat(to_cat, dim=1)
        
    elif name == 'yolo':
        # input size: (B, (4+1+classes)*num_achors=255, Gi, Gi)
        print(x.shape)
        B, C, w, h = x.size()
        # read layer's info
        anchors_list = layer[0].anchors
        classes = layer[0].classes
        in_width = layer[0].in_width
        num_anchs = len(anchors_list)
        # bbox coords + obj score + class scores
        num_feats = 4 + 1 + classes
        
        # transform the predictions
        # (B, ((4+1+classes)*num_achors), Gi, Gi)
        # -> (B, Gi*Gi*num_anchors, (4+1+classes))
        x = x.view(B, num_anchs, num_feats, w, h)
        x = x.permute(0, 3, 4, 1, 2).contiguous() # (B, w, h, num_anchs, num_feats)
        x = x.view(B, h*w*num_anchs, num_feats)
        
        # To calc predictions, first we need to add a center offset (cx, cy).
        # To achieve this we need to create two columns with every possible 
        # combination of two numbers from 0 to num_grid and
        # then add it to the center offset predictions at 0 and 1 indices
        # in x.
        grid = torch.arange(w)
        a, b = torch.meshgrid(grid, grid)
        cx = a.type(torch.FloatTensor).view(-1, 1) # 0, 0, ..., 12, 12
        cy = b.type(torch.FloatTensor).view(-1, 1) # 0, 1, ..., 11, 12
        # cxy.shape is (1, *, 2) where (:, :, 0) is an offset for cx, (:, :, 1) -- for cy
        cxy = torch.cat((cx, cy), dim=1).repeat(1, num_anchs).view(-1, 2).unsqueeze(0)
        cxy = cxy.to(device)
        
        # to calc the offsets for bbox size we need to scale anchors
        stride = in_width // w
        anchors_list = [(anchor[0] / stride, anchor[1] / stride) for anchor in anchors_list]
        anchors_tens = torch.FloatTensor(anchors_list)
        # pwh.shape is the same as cxy
        pwh = anchors_tens.repeat(w * w, 1).unsqueeze(0)
        pwh = pwh.to(device)
        
        # transform the predictions
        x[:, :, 0:2] = torch.sigmoid(x[:, :, 0:2]) + cxy
        x[:, :, 2:4] = pwh * torch.exp(x[:, :, 2:4])
        x[:, :, 4] = torch.sigmoid(x[:, :, 4]) * stride
        x[:, :, 5:5+classes] = torch.sigmoid((x[:, :, 5:5+classes]))
        
        # add new predictions to the list of predictions from all scales
        try:
#             print(x.shape, predictions.shape)
            predictions = torch.cat((predictions, x), dim=1)

        except NameError:
            predictions = x
            
    outputs.append(x)
        
predictions

0 convolutional
torch.Size([2, 3, 608, 608]) Sequential(
  (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky_0): LeakyReLU(negative_slope=0.1)
)
1 convolutional
torch.Size([2, 32, 608, 608]) Sequential(
  (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (bn_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky_1): LeakyReLU(negative_slope=0.1)
)
2 convolutional
torch.Size([2, 64, 304, 304]) Sequential(
  (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
  (bn_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (leaky_2): LeakyReLU(negative_slope=0.1)
)
3 convolutional
torch.Size([2, 32, 304, 304]) Sequential(
  (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn_3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

RuntimeError: Given groups=1, weight of size [256, 1024, 1, 1], expected input[2, 512, 19, 19] to have 1024 channels, but got 512 channels instead

In [ ]:
self_layer_list[0]

In [120]:
# TODO proper transformation
a = torch.arange(1*(80+5)*3*5*5).view(1, 255, 5, 5)
# print(a)
print(a[:, :, 0, 1])
a = a.view(1, 3, 85, 5, 5).permute(0, 3, 4, 1, 2).contiguous().view(1, 75, 85)
print(a[:, 5, :])

tensor([[   1,   26,   51,   76,  101,  126,  151,  176,  201,  226,  251,  276,
          301,  326,  351,  376,  401,  426,  451,  476,  501,  526,  551,  576,
          601,  626,  651,  676,  701,  726,  751,  776,  801,  826,  851,  876,
          901,  926,  951,  976, 1001, 1026, 1051, 1076, 1101, 1126, 1151, 1176,
         1201, 1226, 1251, 1276, 1301, 1326, 1351, 1376, 1401, 1426, 1451, 1476,
         1501, 1526, 1551, 1576, 1601, 1626, 1651, 1676, 1701, 1726, 1751, 1776,
         1801, 1826, 1851, 1876, 1901, 1926, 1951, 1976, 2001, 2026, 2051, 2076,
         2101, 2126, 2151, 2176, 2201, 2226, 2251, 2276, 2301, 2326, 2351, 2376,
         2401, 2426, 2451, 2476, 2501, 2526, 2551, 2576, 2601, 2626, 2651, 2676,
         2701, 2726, 2751, 2776, 2801, 2826, 2851, 2876, 2901, 2926, 2951, 2976,
         3001, 3026, 3051, 3076, 3101, 3126, 3151, 3176, 3201, 3226, 3251, 3276,
         3301, 3326, 3351, 3376, 3401, 3426, 3451, 3476, 3501, 3526, 3551, 3576,
         3601, 3626, 3651, 3